## Data Preparation

In [32]:
import re
import os
from random import choice
from os import listdir
import numpy as np
import pypcd

In [25]:
# currently in submap registration repo
txt_path = "./bin/loop_closures.txt"
data_path = "./data_pcd"
bin_path = "./data_bin"
if not os.path.exists(bin_path):
    os.makedirs(bin_path)
triplet_path = "./triplets.txt"

### Save pcd as bin

In [17]:
data_idx = []  # indices of .pcd files
for string in listdir(data_path):    
    data_idx.append(int(re.findall(r"\d+", string)[0]))
max_idx, min_idx = max(data_idx), min(data_idx)
print max_idx, min_idx

690 0


In [55]:
# need 10 minutes maybe
for idx in range(min_idx, max_idx+1):
    pc = pypcd.PointCloud.from_path(data_path + '/submap_' + str(idx) + '.pcd')
    pc_array = np.array(pc.pc_data)
    # output: N x 3
    np.array(list(map(lambda x: list(x), pc_array))).astype("float32").tofile(bin_path + "/submap_" + str(idx) + '.bin')

Test: read bin file (AND RESHAPE!)

In [64]:
model = np.fromfile(bin_path + "/submap_0.bin", dtype=np.float32)
model = np.reshape(model, (-1, 3))
print model.shape

(77969, 3)


### Generate Triplets

In [65]:
with open(txt_path) as file:
    lines = file.readlines()
    lines = [line.rstrip() for line in lines]
    lines = [line.split(" ") for line in lines]
    lines = [list(map(lambda x: int(x), line)) for line in lines]

    lines: [[anchor, pos1, pos2, ...], 
            [anchor, pos1, pos2, ...], ...
            ]

In [66]:
max_idx_overlap = max(map(max, lines))
min_idx_overlap = min(map(min, lines))
print len(lines), max_idx_overlap, min_idx_overlap

409 689 27


In [67]:
twins = []  # check how many (anchor, positive)
# start_idx = lines[0][0]
# end_idx = lines[-1][0]

for line in lines:
    anchor_idx = line[0]
    for i in range(1, len(line)):
        twins.append((anchor_idx, line[i])) 

print len(twins)

1186


This could be a triplet dataset. But not in the same format as 3DFeatNet.

In [79]:
# triplets = []  # 3 negative for each triplets?
# neg_num = 3

# for line in lines:
#     anchor_idx = line[0]
#     for i in range(1, len(line)):  # each positive one
#         pos = line[i]
#         for neg in range(neg_num):  # more negative ones
#             neg = choice([i for i in range(min_idx, max_idx+1) if i not in line])
#             triplets.append((anchor_idx, pos, neg))
            
# print len(triplets)

This should be correct, however, without "nonnegatives".

In [77]:
triplets_str = []

for line in lines:
    anchor_path = "./data_bin/submap_" + str(line[0]) + ".bin"
    positives = " ".join(list(map(str, line[1:])))
    triplets_str.append(anchor_path + "\t|\t" + positives + "\t| ")

In [78]:
with open(bin_path + "/train.txt", "w") as file:
    file.write("\n".join(triplets_str))